## Introduction

In this project, working with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information.

In [110]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

Introduced the name of the table and what the table looked like in the "factbook.db" database. Run the queries below to get this information directly

In [111]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type = 'table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


For the next run the query to return information on the tables in the database and show first five rows to explore database.

## Overview of the Data

In [112]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

 - name --the name of the country.
 - area --the country's total area (both land and water).
 - area_land --the country's land area in square kilometers.
 - area_water --the country's waterarea in square kilometers.
 - population --the country's population.
 - population_growth --the country's population growth as a percentage.
 - birth_rate --the country's birth rate, or the number of births per year per 1,000 people.
 - death_rate --the country's death rate, or the number of death per year per 1,000 people.

## Summary Statistics

In [113]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts;

Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


After summarize statistics calculation with facts table

 - There's some country with population of 0.
 - There's some country with population 7256490011 (7.2 Billion people)
 - There's some country with population growth of 0.
 - There's some country with population growth of 4.02

## Exploring Outliers

In [114]:
%%sql
SELECT name, population, birth_rate, death_rate
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts);

Done.


name,population,birth_rate,death_rate
Antarctica,0,None,None


Try to figure out what the country that have 0 population and finally found that country is 'Antarctica'!!

In [115]:
%%sql
SELECT name, population, birth_rate, death_rate
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts);

Done.


name,population,birth_rate,death_rate
World,7256490011,18.6,7.8


And try to figure out what the country that have 7.2+ Billion people and found that country is the total result from the whole world!!

Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.

## Summary Statistics Revisited

In [116]:
%%sql
SELECT MIN(population),
       MAX(population),
       MIN(population_growth),
       MAX(population_growth)
  FROM facts
 WHERE name <> 'World';

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!

In [117]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts
                      WHERE name <> 'World');
                

Done.


name
China


Ofcourse, China is the answer!

## Exploring Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

In [118]:
%%sql
-- Find out the average of population and the country's area
SELECT ROUND(AVG(population), 2) AS avg_pop,
       ROUND(AVG(area), 2) AS avg_area
  FROM facts
 WHERE name <> 'World';

Done.


avg_pop,avg_area
32242666.57,555093.55


In [119]:
%%sql
/* 1) Find out the country that have the population is above average.
   2) Find out the country that have the area is below average.
*/

SELECT *
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name <> 'World'
                    )
       
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name <> 'World'
              
 LIMIT 10);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


These country in the table above is the country that have density from both the number of population and number of country's area.

But, Actually need to recalculate with the "area_land" column with the other way and the people live in the land area not water area.

In [120]:
%%sql
SELECT name, population, area_land,
       population / area_land AS pop_per_area
  FROM facts
 WHERE name <> 'World'
 ORDER BY pop_per_area DESC
 LIMIT 10;

Done.


name,population,area_land,pop_per_area
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,687,8259
Hong Kong,7141106,1073,6655
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


Figure out that "Macau" is the country that have the most density population per land area and "Monaco", "Singapore" and "Hong Kong" respectively.

## Exploring which country have the highest growth rate

In [121]:
%%sql
SELECT name, population_growth
  FROM facts
 WHERE name <> 'World'
 ORDER BY population_growth DESC
 LIMIT 10;

Done.


name,population_growth
South Sudan,4.02
Malawi,3.32
Burundi,3.28
Niger,3.25
Uganda,3.24
Qatar,3.07
Burkina Faso,3.03
Mali,2.98
Cook Islands,2.95
Iraq,2.93


Figure out that "South Sudan" is the country that have to most population growth rate that found in the earlier queries.

Let's find which country will add the most people to thier populations next year?

In [122]:
%%sql
SELECT name, population, population_growth,
       population * population_growth AS next_year_pop
  FROM facts
 WHERE name <> 'World'
 ORDER BY next_year_pop DESC
 LIMIT 10;

Done.


name,population,population_growth,next_year_pop
India,1251695584,1.22,1527068612.48
China,1367485388,0.45,615368424.6
Nigeria,181562056,2.45,444827037.20000005
Pakistan,199085847,1.46,290665336.62
Ethiopia,99465819,2.89,287456216.91
Bangladesh,168957745,1.6,270332392.0
United States,321368864,0.78,250667713.92000002
Indonesia,255993674,0.92,235514180.08
"Congo, Democratic Republic of the",79375136,2.45,194469083.20000002
Philippines,100998376,1.61,162607385.36


If multiply the current year of population with population growth in the same year "India" will be the country in the next year that is the most population in the world.

## Exploring which country have a higher death rate than birth rate

In [123]:
%%sql
SELECT name, birth_rate, death_rate,
       ROUND(death_rate - birth_rate, 2) AS diff
  FROM facts
 WHERE name <> 'World'
   AND birth_rate < death_rate
 ORDER BY diff DESC
 LIMIT 10;

Done.


name,birth_rate,death_rate,diff
Bulgaria,8.92,14.44,5.52
Serbia,9.08,13.66,4.58
Latvia,10.0,14.31,4.31
Lithuania,10.1,14.27,4.17
Ukraine,10.72,14.46,3.74
Hungary,9.16,12.73,3.57
Germany,8.47,11.42,2.95
Slovenia,8.42,11.37,2.95
Romania,9.14,11.9,2.76
Croatia,9.45,12.18,2.73


Another topic that is which countries that have a higher death rate than birth rate!!

For the answer is "Bulgaria"

## Exploring about water area and land area
1) which countries have the highest ratios of water to land? 

2) which countries have more water than land?

Let's explore the data..

In [124]:
%%sql
SELECT name, area_water
  FROM facts
 WHERE name <> 'World'
 ORDER BY area_water DESC
 LIMIT 10;

Done.


name,area_water
Canada,891163
Russia,720500
United States,664709
India,314070
China,270550
Brazil,157630
Iran,116600
Ethiopia,104300
Colombia,100210
Indonesia,93000


In [125]:
%%sql
SELECT MIN(area_water),
       MAX(area_water)
  FROM facts
 WHERE name <> 'World'

Done.


MIN(area_water),MAX(area_water)
0,891163


There's some countries with area water with 0.

Let's explore more to find insight.

In [126]:
%%sql
SELECT name, area_water
  FROM facts
 WHERE area_water = (SELECT MIN(area_water)
                       FROM facts
                      WHERE name <> 'World')
 LIMIT 10;

Done.


name,area_water
Afghanistan,0
Algeria,0
Andorra,0
Angola,0
Antigua and Barbuda,0
Bahrain,0
Barbados,0
Bhutan,0
Cabo Verde,0
Central African Republic,0


In [127]:
%%sql
SELECT COUNT(*) AS no_water_area
  FROM facts
 WHERE area_water = (SELECT MIN(area_water)
                       FROM facts
                      WHERE name <> 'World');

Done.


no_water_area
90


There are 90 countries that don't have area water in thier countries.

In [128]:
%%sql
SELECT name, area_land
  FROM facts
 WHERE name <> 'World'
 ORDER BY area_land DESC
 LIMIT 10;

Done.


name,area_land
Russia,16377742
China,9326410
United States,9161966
Canada,9093507
Brazil,8358140
Australia,7682300
India,2973193
Argentina,2736690
Kazakhstan,2699700
Algeria,2381741


### Conclude

 - For the country that have the most area of land is "Russia".
 - For the country that have the most area of water is "Canada".
 
Let's find out which countries that have area of water than land

In [129]:
%%sql
SELECT name,
       area_water - area_land AS diff_area
  FROM facts
 WHERE name <> 'World'
 ORDER BY diff_area DESC
 LIMIT 10;

Done.


name,diff_area
British Indian Ocean Territory,54280
Virgin Islands,1218
Holy See (Vatican City),0
Monaco,-2
Coral Sea Islands,-3
Ashmore and Cartier Islands,-5
Navassa Island,-5
Spratly Islands,-5
Clipperton Island,-6
Gibraltar,-6


For the answer is "British Indian Ocean Territory" is the country that have more water than land.